In [155]:
import pandas as pd
import numpy as np
from src.utils.json_utils import read_json_lines
import os
import re
import ast
import copy
cur_dir = os.getcwd()
print(cur_dir)

/home/c_spino/comp_550/comp-550-project/notebooks


# Smokers data distribution

In [110]:
train = read_json_lines(os.path.join(cur_dir,"..", "data/smokers/augmentation/train.json"))
val = read_json_lines(os.path.join(cur_dir, "..", "data/smokers/augmentation/validation.json"))
test = read_json_lines(os.path.join(cur_dir, "..", "data/smokers/augmentation/test.json"))
train = pd.DataFrame(data={"label": [x["label"] for x in train], "text": [x["text"] for x in train]})
val = pd.DataFrame(data={"label": [x["label"] for x in val], "text": [x["text"] for x in val]})
test = pd.DataFrame(data={"label": [x["label"] for x in test], "text": [x["text"] for x in test]})
all_data = pd.concat([train, val, test])
SMOKERS = {
    "unknown": 0,
    "non-smoker": 1,
    "past-smoker": 2,
    "smoker": 3,
    "current-smoker": 4,
}
SMOKERS_INVERSE = dict(zip(SMOKERS.values(), SMOKERS.keys()))
all_data.groupby("label").size()
all_data.replace(SMOKERS_INVERSE).groupby("label").size()


label
current-smoker     46
non-smoker         82
past-smoker        47
smoker             12
unknown           315
dtype: int64

# DA Performance for BoW models
Manual extraction from the logs is required for the test $F_1$.

In [180]:
# Compute the macro average f1 based on confusion matrix
def compute_f1(confusion_matrix):
    TP = np.diag(confusion_matrix)
    FP = np.sum(confusion_matrix, axis=0) - TP
    FN = np.sum(confusion_matrix, axis=1) - TP
    num_classes = confusion_matrix.shape[0]
    TN = []
    for i in range(num_classes):
        temp = np.delete(confusion_matrix, i, 0)  # delete ith row
        temp = np.delete(temp, i, 1)  # delete ith column
        TN.append(sum(sum(temp)))
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F1 = 2*(precision*recall)/(precision+recall)
    F1 = np.nan_to_num(F1)
    macro_f1 = F1.mean()
    return macro_f1

In [283]:
# Read the performance
polarity_results = read_json_lines(os.path.join(cur_dir, "..", "logs/training/polarity_constant.json"))
articles_results = read_json_lines(os.path.join(cur_dir, "..", "logs/training/articles_constant.json"))
smokers_results = read_json_lines(os.path.join(cur_dir, "..", "logs/training/smokers_constant.json"))
# Convert to dataframe
polarity_results = pd.DataFrame(polarity_results)
articles_results = pd.DataFrame(articles_results)
smokers_results = pd.DataFrame(smokers_results)
# Drop the pickle path
polarity_results.drop(columns=["path_to_pickle"], inplace=True)
articles_results.drop(columns=["path_to_pickle"], inplace=True)
smokers_results.drop(columns=["path_to_pickle"], inplace=True)
data_types = ["polarity", "articles", "smokers"]
reshape_size = {"polarity": 2, "articles": 5, "smokers": 3}
data_dict_svm = {}
for data_type in data_types:
    # Add the test f1 from the logs using re
    # (because I mistakenly did not add these to the json!!!!)
    with open(os.path.join(cur_dir, "..", f"logs/training/{data_type}_constant.log"), "r") as f:
        text = f.read()
        results = re.findall(r"(\[\[(.*\n\s.*)*\]\])", text)
    results = [x[0] for x in results]
    results = [x.replace("\n ", " ") for x in results]
    results = [x.replace("[", "").replace("]", "") for x in results]
    results = [np.fromstring(x, dtype=np.int64, sep=" ").reshape(-1, reshape_size[data_type]) for x in results]
    # Get the aug info
    data = []
    with open(os.path.join(cur_dir, "..", f"logs/training/{data_type}_constant.log"), "r") as f:
        dict_ = {}
        for i, line in enumerate(f):
            if "with augmentation" in line.lower():
                result = re.match(r".*with augmentation (\d+)", line)
                dict_["augmentation_id"] = int(result.group(1))
            if "augmentation features" in line.lower():
                result = re.match(r".*Augmentation features: (.*)", line)
                dict_["augmentation_features"] = ast.literal_eval(result.group(1))
                data.append(dict_)
                dict_ = {}
    # Get the test f1
    for i in range(len(data)):
        data[i]["nb_f1"] = compute_f1(results[6*i + 1])
        data[i]["lr_f1"] = compute_f1(results[6*i + 3])
        data[i]["svm_f1"] = compute_f1(results[6*i + 5])
    data = pd.DataFrame(data)
    data_dict_svm[data_type] = data

/home/c_spino/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
/home/c_spino/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [284]:
data_dict_svm["polarity"]["svm_f1_round"] = np.round(data_dict_svm["polarity"]["svm_f1"], decimals=3)
data_dict_svm["polarity"]

,augmentation_id,augmentation_features,nb_f1,lr_f1,svm_f1,svm_f1_round
0,0,"{'num_samples': 0, 'augmentation_type': 'none'}",0.804025,0.766238,0.768327,0.768
1,1,"{'num_samples': 5, 'augmentation_type': 'rando...",0.782484,0.775894,0.783334,0.783
2,2,"{'num_samples': 5, 'augmentation_type': 'rando...",0.779728,0.775806,0.788939,0.789
3,3,"{'num_samples': 5, 'augmentation_type': 'rando...",0.789984,0.775929,0.785331,0.785
4,4,"{'num_samples': 5, 'augmentation_type': 'rando...",0.773090,0.768246,0.792667,0.793
5,5,"{'num_samples': 5, 'augmentation_type': 'rando...",0.785252,0.765589,0.778763,0.779
6,6,"{'num_samples': 5, 'augmentation_type': 'rando...",0.789092,0.775065,0.766595,0.767
7,7,"{'num_samples': 5, 'augmentation_type': 'synon...",0.783468,0.767465,0.782499,0.782
8,8,"{'num_samples': 5, 'augmentation_type': 'synon...",0.790859,0.763714,0.783444,0.783
9,9,"{'num_samples': 5, 'augmentation_type': 'synon...",0.781569,0.758068,0.774105,0.774


In [285]:
data_dict_svm['articles']["svm_f1_round"] = np.round(data_dict_svm['articles']["svm_f1"], decimals=3)
data_dict_svm['articles']

,augmentation_id,augmentation_features,nb_f1,lr_f1,svm_f1,svm_f1_round
0,0,"{'num_samples': 0, 'augmentation_type': 'none'}",0.939534,0.953497,0.961704,0.962
1,1,"{'num_samples': 5, 'augmentation_type': 'rando...",0.946347,0.962548,0.957285,0.957
2,2,"{'num_samples': 5, 'augmentation_type': 'rando...",0.956668,0.962548,0.957285,0.957
3,3,"{'num_samples': 5, 'augmentation_type': 'rando...",0.954448,0.962548,0.957285,0.957
4,4,"{'num_samples': 5, 'augmentation_type': 'rando...",0.950697,0.961704,0.955707,0.956
5,5,"{'num_samples': 5, 'augmentation_type': 'rando...",0.950697,0.957017,0.946448,0.946
6,6,"{'num_samples': 5, 'augmentation_type': 'rando...",0.954976,0.955997,0.955883,0.956
7,7,"{'num_samples': 5, 'augmentation_type': 'synon...",0.950739,0.966276,0.966276,0.966
8,8,"{'num_samples': 5, 'augmentation_type': 'synon...",0.942296,0.961584,0.961584,0.962
9,9,"{'num_samples': 5, 'augmentation_type': 'synon...",0.938451,0.946606,0.970874,0.971


In [286]:
data_dict_svm["smokers"]["svm_f1_round"] = np.round(data_dict_svm["smokers"]["svm_f1"], decimals=3)
data_dict_svm["smokers"]

,augmentation_id,augmentation_features,nb_f1,lr_f1,svm_f1,svm_f1_round
0,0,"{'num_samples': 0, 'augmentation_type': 'none'}",0.247934,0.383668,0.247934,0.248
1,1,"{'num_samples': 5, 'augmentation_type': 'rando...",0.247934,0.528260,0.475498,0.475
2,2,"{'num_samples': 5, 'augmentation_type': 'rando...",0.247934,0.549317,0.521787,0.522
3,3,"{'num_samples': 5, 'augmentation_type': 'rando...",0.247934,0.549317,0.531469,0.531
4,4,"{'num_samples': 5, 'augmentation_type': 'rando...",0.247934,0.562435,0.542781,0.543
5,5,"{'num_samples': 5, 'augmentation_type': 'rando...",0.247934,0.550293,0.525528,0.526
6,6,"{'num_samples': 5, 'augmentation_type': 'rando...",0.247934,0.525528,0.314204,0.314
7,7,"{'num_samples': 5, 'augmentation_type': 'synon...",0.247934,0.536862,0.475498,0.475
8,8,"{'num_samples': 5, 'augmentation_type': 'synon...",0.247934,0.522268,0.483896,0.484
9,9,"{'num_samples': 5, 'augmentation_type': 'synon...",0.247934,0.509158,0.393385,0.393


# DA performance for LSTM

In [287]:
data_types = ["polarity", "articles", "smokers"]
model_type = {
    "polarity": ["distilbert-base-uncased"],
    "articles": ["distilbert-base-uncased"],
    "smokers": [
        "distilbert-base-uncased",
        "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
    ],
}
data_dict_lstm = {}
for data_type in data_types:
    # Add the test f1 from the logs using re
    # (because I mistakenly did not add these to the json!!!!)
    with open(os.path.join(cur_dir, "..", f"logs/training_rnn_classif/{data_type}.log"), "r") as f:
        text = f.read()
        results = re.findall(r"Here is the test results.*\n.*\n.*f1 score.*(0\.\d+)", text)
    # Get the aug info
    data = []
    with open(os.path.join(cur_dir, "..", f"logs/training_rnn_classif/{data_type}.log"), "r") as f:
        dict_ = {}
        for i, line in enumerate(f):
            if "with augmentation" in line.lower():
                result = re.match(r".*with augmentation (\d+)", line)
                dict_["augmentation_id"] = int(result.group(1))
            if "augmentation features" in line.lower():
                result = re.match(r".*Augmentation features: (.*)", line)
                dict_["augmentation_features"] = ast.literal_eval(result.group(1))
                data.append(dict_)
                dict_ = {}
    data_new = copy.deepcopy(data)
    if data_type == "smokers":
        data_new = [copy.deepcopy(x) for x in data_new for _ in range(2)]
        for i, dict_ in enumerate(data_new):
            if (i % 2) == 0:
                data_new[i]["model_type"] = "distilbert-base-uncased"
                data_new[i]["f1"] = results[i]
            else:
                data_new[i]["model_type"] = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
                data_new[i]["f1"] = results[i]
    else:
        for i, dict_ in enumerate(data_new):
            data_new[i]["model_type"] = "distilbert-base-uncased"
            data_new[i]["f1"] = results[i]
    data = pd.DataFrame(data_new)
    data_dict_lstm[data_type] = data

In [288]:
data_dict_lstm["polarity"]
# NOTE: The values for augmentation_id = {1, 2, 3} are recalculated because they were initially on
# the unidirectional LSTM model
data_dict_lstm["polarity"].loc[data_dict_lstm["polarity"]["augmentation_id"] == 1, "f1"] = 0.730
data_dict_lstm["polarity"].loc[data_dict_lstm["polarity"]["augmentation_id"] == 2,"f1"] = 0.722
data_dict_lstm["polarity"].loc[data_dict_lstm["polarity"]["augmentation_id"] == 3, "f1"] = 0.730
data_dict_lstm["polarity"]

,augmentation_id,augmentation_features,model_type,f1
0,0,"{'num_samples': 0, 'augmentation_type': 'none'}",distilbert-base-uncased,0.771
1,1,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.73
2,2,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.722
3,3,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.73
4,4,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.749
5,5,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.732
6,6,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.745
7,7,"{'num_samples': 5, 'augmentation_type': 'synon...",distilbert-base-uncased,0.770
8,8,"{'num_samples': 5, 'augmentation_type': 'synon...",distilbert-base-uncased,0.772
9,9,"{'num_samples': 5, 'augmentation_type': 'synon...",distilbert-base-uncased,0.759


In [289]:
data_dict_lstm["articles"]
# NOTE: The values for augmentation_id = {5, 6, 9, 11, 12} are recalculated because they were initially on
# the unidirectional LSTM model
data_dict_lstm["articles"].loc[data_dict_lstm["articles"]["augmentation_id"] == 5, "f1"] = 0.937
data_dict_lstm["articles"].loc[data_dict_lstm["articles"]["augmentation_id"] == 6, "f1"] = 0.926
data_dict_lstm["articles"].loc[data_dict_lstm["articles"]["augmentation_id"] == 9, "f1"] = 0.914
data_dict_lstm["articles"].loc[data_dict_lstm["articles"]["augmentation_id"] == 11, "f1"] = 0.924
data_dict_lstm["articles"].loc[data_dict_lstm["articles"]["augmentation_id"] == 12, "f1"] = 0.924
data_dict_lstm["articles"]

,augmentation_id,augmentation_features,model_type,f1
0,0,"{'num_samples': 0, 'augmentation_type': 'none'}",distilbert-base-uncased,0.888
1,1,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.892
2,2,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.906
3,3,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.930
4,4,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.924
5,5,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.937
6,6,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.926
7,7,"{'num_samples': 5, 'augmentation_type': 'synon...",distilbert-base-uncased,0.914
8,8,"{'num_samples': 5, 'augmentation_type': 'synon...",distilbert-base-uncased,0.928
9,9,"{'num_samples': 5, 'augmentation_type': 'synon...",distilbert-base-uncased,0.914


In [290]:
data_dict_lstm["smokers"]

,augmentation_id,augmentation_features,model_type,f1
0,0,"{'num_samples': 0, 'augmentation_type': 'none'}",distilbert-base-uncased,0.365
1,0,"{'num_samples': 0, 'augmentation_type': 'none'}",microsoft/BiomedNLP-PubMedBERT-base-uncased-ab...,0.345
2,1,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.350
3,1,"{'num_samples': 5, 'augmentation_type': 'rando...",microsoft/BiomedNLP-PubMedBERT-base-uncased-ab...,0.361
4,2,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.284
5,2,"{'num_samples': 5, 'augmentation_type': 'rando...",microsoft/BiomedNLP-PubMedBERT-base-uncased-ab...,0.444
6,3,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.425
7,3,"{'num_samples': 5, 'augmentation_type': 'rando...",microsoft/BiomedNLP-PubMedBERT-base-uncased-ab...,0.386
8,4,"{'num_samples': 5, 'augmentation_type': 'rando...",distilbert-base-uncased,0.354
9,4,"{'num_samples': 5, 'augmentation_type': 'rando...",microsoft/BiomedNLP-PubMedBERT-base-uncased-ab...,0.376


# Average improvement of standard DA technique for each classifier $\times$ task pair

In [301]:
# For the svm models
average_delta_f1 = []
best_p_count = [0, 0, 0]
for data_type in ["polarity", "articles", "smokers"]:
    data_dict_svm[data_type]["augmentation_type"] = data_dict_svm[data_type]["augmentation_features"].apply(lambda x: x["augmentation_type"])
    # Take the max because that's what you would pick for the DA
    # Also calculate what's the best p_count
    f1_0 = data_dict_svm[data_type].loc[0, "svm_f1_round"]
    f1_list = data_dict_svm[data_type].loc[1:, "svm_f1_round"].values
    best_p_list = np.array([f1_list[i:i+3].argmax() for i in range(0, len(f1_list), 3)])
    best_p_count = [x + (best_p_list == i).sum() for i, x in enumerate(best_p_count)]
    f1_list = np.array([f1_list[i:i+3].max() for i in range(0, len(f1_list), 3)])
    print(data_type, f1_list)
    average_delta_f1.append(((f1_list - f1_0)/f1_0).mean())
print(np.array(average_delta_f1)*100)
print(best_p_list)
print(best_p_count)
# print(f1_0, f1_list)
# data_dict_svm["smokers"]

polarity [0.789 0.793 0.783 0.79 ]
articles [0.957 0.956 0.971 0.971]
smokers [0.531 0.543 0.484 0.484 0.464]
[  2.70182292   0.18191268 102.09677419]
[2 0 1 0 1]
[5, 5, 3]


In [306]:
# For the svm models
average_delta_f1 = []
best_p_count = [0, 0, 0]
model_type = {
    "polarity": ["distilbert-base-uncased"],
    "articles": ["distilbert-base-uncased"],
    "smokers": [
        "distilbert-base-uncased",
        "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
    ],
}
for data_type in ["polarity", "articles", "smokers"]:
    for model in model_type[data_type]:
        data_ = data_dict_lstm[data_type]
        data_ = data_[data_["model_type"] == model]
        # data_["augmentation_type"] = data_["augmentation_features"].apply(lambda x: x["augmentation_type"])
        # Take the max because that's what you would pick for the DA
        f1_0 = float(data_["f1"].values[0])
        f1_list = data_.loc[1:, "f1"].values.astype(float)
        best_p_list = np.array([f1_list[i:i+3].argmax() for i in range(0, len(f1_list), 3)])
        best_p_count = [x + (best_p_list == i).sum() for i, x in enumerate(best_p_count)]
        f1_list = np.array([f1_list[i:i+3].max() for i in range(0, len(f1_list), 3)])
        print(data_type, model, f1_list)
        average_delta_f1.append(((f1_list - f1_0)/f1_0).mean())
print(np.array(average_delta_f1)*100)
print(best_p_count)

polarity distilbert-base-uncased [0.73  0.749 0.772 0.772]
articles distilbert-base-uncased [0.93  0.937 0.928 0.937]
smokers distilbert-base-uncased [0.425 0.408 0.385 0.424 0.422]
smokers microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext [0.444 0.386 0.505 0.438 0.432 0.378]
[-1.97795071  5.06756757 13.09589041 24.7826087 ]
[5, 6, 8]


# Comparing our model to the BERT augmentation

In [309]:
df = pd.DataFrame(data={
    "random_mask": [0.772, 0.945, 0.424, 0.422, 0.423, 0.432], 
    "our_model": [0.783, 0.95, 0.451, 0.451, 0.537, 0.537]
})

In [314]:
delta_increase = (((df["our_model"] - df["random_mask"])/df["random_mask"])*100).values

In [316]:
delta_increase[2:4].mean()

6.6199812214969205

In [317]:
delta_increase[4:].mean()


25.627955082742325